In [53]:
from app.main.func import db_filter_req

curriculum_discipline_id = 10464
data = db_filter_req("mm_work_programs", "curriculum_discipline_id", curriculum_discipline_id)
data

[{'id': '3229',
  'curriculum_discipline_id': '10464',
  'name': 'Основы управления в органах внутренних дел',
  'description': None,
  'authors': '',
  'reviewers_int': None,
  'reviewers_ext': 'Test',
  'date_create': None,
  'date_approval': '2016-11-11',
  'date_department': '2021-12-14',
  'document_department': '1111',
  'date_methodical': '2021-11-13',
  'document_methodical': '5',
  'date_academic': '2021-11-13',
  'document_academic': '8',
  'status': None,
  'user_id': '9',
  'settings': '{"printSettingsRange":""}'}]

In [71]:
work_program_id = 3229
data = db_filter_req("mm_sections", "work_program_id", work_program_id)
data

[{'id': '1906',
  'work_program_id': '3229',
  'topic_id': None,
  'purposes': 'Цели',
  'tasks': 'Задачи',
  'place_in_structure': 'Место в структуре',
  'knowledge': '',
  'skills': '',
  'abilities': '',
  'ownerships': ''}]

In [4]:
work_program_id = 3229
data = db_filter_req("mm_work_programs_data", "work_program_id", work_program_id)
data

[{'id': '42316',
  'work_program_id': '3229',
  'field_id': '1',
  'data': 'ТЕСТ\n',
  'check': '1'},
 {'id': '42317',
  'work_program_id': '3229',
  'field_id': '2',
  'data': '1. Анисина, С. С. Организация планирования в органах внутренних дел Российской Федерации [Электронный ресурс]: учебно-практическое пособие / С. С. Анисина, Э. Г. Юзиханова. - Тюмень: ТИПК МВД России, 2018. - 106 с. - URL: http://www.liborui.ru. - Режим доступа: для авторизир. пользователей\n2. Бавсун, И. Г. Основы управления в органах внутренних дел [Электронный ресурс]: учебное пособие / И. Г. Бавсун. - Омск: Омская академия МВД России, 2017. - 152 с. - URL: http://www.liborui.ru. - Режим доступа: для авторизир. пользователей\n3. Власов, Б. Е. Программно-целевое управление в органах внутренних дел [Электронный ресурс]: учебное пособие / Б. Е. Власов. - М.: АУ МВД России, 2018. - 120 с. - URL: http://www.liborui.ru. - Режим доступа: для авторизир. пользователей\n4. Михайлов, В. И. Функции руководителя по управл

In [6]:
work_program_id = 3229
data = db_filter_req("mm_work_programs_competencies_data", "work_program_id", work_program_id)
data

[{'id': '19938',
  'work_program_id': '3229',
  'competency_id': '4565',
  'field_id': '1',
  'value': '1'}]

In [18]:
import requests
from app.main.func import db_filter_req, plan_curriculum_disciplines
from config import ApeksAPI
from app.programs.func import plan_department_disciplines

class WorkProgram:
    def __init__(self, curriculum_discipline_id):
        self.curriculum_discipline_id = curriculum_discipline_id
        self.mm_work_programs = db_filter_req(
            "mm_work_programs", "curriculum_discipline_id", curriculum_discipline_id
        )
        self.work_program_id = self.mm_work_programs[0]["id"]
        self.name = self.mm_work_programs[0]["name"]

        self.mm_work_programs_items = {
            "name": "name",  # Название программы
            "reviewers_ext": "reviewers_ext",  # Рецензенты
            "date_approval": "date_approval",
            "date_methodical": "date_methodical",
            "document_methodical": "document_methodical",
            "date_academic": "date_academic",
            "document_academic": "document_academic",
        }
        self.special = {
            "date_department": "date_department",  # Дата заседания кафедры
            "document_department": "document_department",  # Протокол заседания кафедры
        }


        self.mm_sections_items = {
            "purposes": "purposes",  # Цели
            "tasks": "tasks",  # Задачи
            "place_in_structure": "place_in_structure",  # Место в структуре ООП
        }
        self.mm_work_programs_data_items = {
            "authorprint": 29,  # Автор(ы) рабочей программы (для печати)
            "no_next_disc": 30,  # Пояснение к таблице с последующими дисциплинами (информация об отсутствии)
            "templan_info": 31,  # Примечание к тематическому плану
            "self_provision": 8,  # Обеспечение самостоятельной работы
            "test_criteria": 32,  # Критерии оценки для сдачи промежуточной аттестации в форме тестирования
            "course_works": 12,  # Тематика курсовых работ
            "practice": 10,  # Практикум
            "control_works": 13,  # Тематика контрольных работ
            "exam_form_desc": 33,  # Примерные оценочные средства для проведения промежуточной аттестации
            "task_works": 9,  # Задачи
            "tests": 19,  # Тесты
            "regulations": 16,  # Нормативные акты
            "library_main": 1,  # Основная литература
            "library_add": 2,  # Дополнительная литература
            "library_np": 3,  # Научная продукция
            "internet": 18,  # Ресурсы информационно-телекоммуникационной сети Интернет
            "software": 15,  # Программное обеспечение
            "ref_system": 17,  # Базы данных, информационно-справочные и поисковые системы
            "materials_base": 20,  # Описание материально-технической базы
        }

    def get(self, parameter):
        if parameter in self.mm_work_programs_items:
            return self.mm_work_programs[0][self.mm_work_programs_items.get(parameter)]
        elif parameter in self.special:
            date_department = self.mm_work_programs[-1]["date_department"]
            document_department = self.mm_work_programs[-1]["document_department"]
            if date_department:
                date = date_department.split("-")
                date_department = f"{date[-1]}.{date[-2]}.{date[-3]}"
            else:
                date_department = "[Не заполнена]"
            if document_department is None:
                document_department = "[Отсутствует]"
            return f"Дата заседания кафедры: {date_department}\r\nПротокол № {document_department}"
        elif parameter in self.mm_sections_items:
            return db_filter_req(
                "mm_sections", "work_program_id", self.work_program_id
            )[-1][self.mm_sections_items.get(parameter)]
        elif parameter in self.mm_work_programs_data_items:
            try:
                params = {
                    "token": ApeksAPI.TOKEN,
                    "table": "mm_work_programs_data",
                    "filter[work_program_id]": self.work_program_id,
                    "filter[field_id]": str(
                        self.mm_work_programs_data_items.get(parameter)
                    ),
                }
                return requests.get(
                    ApeksAPI.URL + "/api/call/system-database/get", params=params
                ).json()["data"][-1]["data"]
            except IndexError:
                return ""
        else:
            return "Wrong parameter"

    def edit(self, parameter, load_data):
        if parameter in self.mm_work_programs_items:
            params = {"token": ApeksAPI.TOKEN}
            data = {
                "table": "mm_work_programs",
                "filter[id]": self.work_program_id,
                "fields["
                + self.mm_work_programs_items.get(parameter)
                + "]": str(load_data),
            }
            requests.post(
                ApeksAPI.URL + "/api/call/system-database/edit",
                params=params,
                data=data,
            )
            self.mm_work_programs = db_filter_req(
                "mm_work_programs",
                "curriculum_discipline_id",
                self.curriculum_discipline_id,
            )

        elif parameter in self.mm_sections_items:
            params = {"token": ApeksAPI.TOKEN}
            data = {
                "table": "mm_sections",
                "filter[work_program_id]": self.work_program_id,
                "fields[" + self.mm_sections_items.get(parameter) + "]": str(load_data),
            }
            r = requests.post(
                ApeksAPI.URL + "/api/call/system-database/edit",
                params=params,
                data=data,
            )
            return r.json()
        elif parameter in self.mm_work_programs_data_items:
            params = {"token": ApeksAPI.TOKEN}
            data = {
                "table": "mm_work_programs_data",
                "filter[work_program_id]": self.work_program_id,
                "filter[field_id]": self.mm_work_programs_data_items.get(parameter),
                "fields[data]": str(load_data),
            }
            r = requests.post(
                ApeksAPI.URL + "/api/call/system-database/edit",
                params=params,
                data=data,
            )
            return r.json()
        else:
            return "Wrong parameter"


class WorkProgramBunchData:
    def __init__(self, parameter, education_plan_id):
        self.parameter = parameter
        self.education_plan_id = education_plan_id

    def all(self):
        disc_list = plan_curriculum_disciplines(self.education_plan_id)
        return self.wp_list_processing(disc_list)

    def department (self, department_id):
        disc_list = plan_department_disciplines(self.education_plan_id, department_id)
        return self.wp_list_processing(disc_list)

    def wp_list_processing(self, disc_list):
        result = {}
        for disc in disc_list:
            try:
                wp = WorkProgram(disc)
            except IndexError:
                result[" ".join(disc_list[disc])] = "-->Программа отсутствует<--"
                break
            try:
                result[" ".join(disc_list[disc])] = wp.get(self.parameter)
            except IndexError:
                result[" ".join(disc_list[disc])] = ""
        return result

In [19]:
test = WorkProgramBunchData('date_department', 49)

In [20]:
test.department(12)

{'Б.1.О.11 Информатика и информационные технологии в профессиональной деятельности': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.В.3 Основы управления в органах внутренних дел': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.В.8 Основы кибербезопасности': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.В.ЭД.2.01 Правовая статистика': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.В.ЭД.4.01 Основы информационной безопасности органов внутренних дел': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.В.ЭД.4.02 Криминальная среда и современные информационные технологии': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11'}

In [21]:
test.all()

{'Б.1.О.1 Иностранный язык': 'Дата заседания кафедры: 10.06.2021\r\nПротокол № 11 ',
 'Б.1.О.2 Философия': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.О.3 Логика': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.О.4 Экономика': 'Дата заседания кафедры: 16.06.2021\r\nПротокол № 14',
 'Б.1.О.5 Политология': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.О.6 Социология': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.О.7 Профессиональная этика и служебный этикет': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.О.8 Психология': 'Дата заседания кафедры: 16.06.2021\r\nПротокол № 15',
 'Б.1.О.9 Русский язык в деловой документации': 'Дата заседания кафедры: 10.06.2021\r\nПротокол № 11',
 'Б.1.О.10 Риторика': 'Дата заседания кафедры: 10.06.2021\r\nПротокол № 11',
 'Б.1.О.11 Информатика и информационные технологии в профессиональной деятельности': 'Дата заседания кафедры: 02.06.2021\r\nПротокол № 11',
 'Б.1.О.12 Теория госуд

In [45]:
plan_department_disciplines(133, 12)

{}

In [84]:


wp = WorkProgram(4553)

In [89]:
wp.get('tasks')

'дать обучающимся знания в области истории и методологии социологической науки, ознакомить их с современными социологическими концепциями и течениями;\r\nсформировать у обучающихся научное социологическое представление об обществе как сложной социокультурной системе, о социальных проблемах современного российского общества и тенденциях его развития;\r\nвыработать уобучающихся понимание значения общественных факторов социализации личности, ее социальных ролей и причин девиантного поведения;\r\nспособствовать формированию у обучающихся понимания причин социальных конфликтов и способов их разрешения в интересах обеспечения общественной безопасности, а также навыков толерантного поведения в многосоставном полиэтничном обществе;\r\nпривить обучающимся элементарные навыки самостоятельного сбора и оценки первичной социологической информации, научного анализа общественных проблем, в том числе связанных с профессиональной деятельностью.'

In [86]:
wp.edit('templan_info', 'Семинарское занятие по теме №2 проводится с использованием активных и интерактивных форм обучения\r\nПрактическое занятие №3 реализуется в форме практической подготовки.')

{'status': 1, 'data': 2}